In [13]:
import pandas as pd 
data=pd.read_csv('/Users/s/Desktop/mlops/network_Security_end2end/artifact/data/processed/train.csv')
group_mapping = {
    'BENIGN': 'Normal Traffic',
    'DoS Hulk': 'DoS',
    'DDoS': 'DDoS',
    'PortScan': 'Port Scanning',
    'DoS GoldenEye': 'DoS',
    'FTP-Patator': 'Brute Force',
    'DoS slowloris': 'DoS',
    'DoS Slowhttptest': 'DoS',
    'SSH-Patator': 'Brute Force',
    'Bot': 'Bots',
    'Web Attack � Brute Force': 'Web Attacks',
    'Web Attack � XSS': 'Web Attacks',
    'Infiltration': 'Infiltration',
    'Web Attack � Sql Injection': 'Web Attacks',
    'Heartbleed': 'Miscellaneous'
}

data['Attack Type'] = data['Label'].map(group_mapping)

In [2]:
data['Attack Type'].value_counts()

Attack Type
Normal Traffic    1062672
DoS                118118
Port Scanning       74299
DDoS                59853
Brute Force          6468
Web Attacks          1020
Bots                  919
Infiltration           17
Miscellaneous           5
Name: count, dtype: int64

In [14]:
data.drop(columns='Label', inplace=True)

In [15]:
X=data.drop(columns=['Attack Type'])
y=data['Attack Type']

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
data['Attack Type'] = le.fit_transform(data['Attack Type'])
data['Attack Type'].value_counts()

Attack Type
6    1062672
3     118118
7      74299
2      59853
1       6468
8       1020
0        919
4         17
5          5
Name: count, dtype: int64

In [17]:
import numpy as np
X = pd.DataFrame(X).fillna(0)
X = np.where(np.isinf(X), 0, X)
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value=0)
X= imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)






In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier

X = pd.DataFrame(X).fillna(0)

X = np.where(np.isinf(X), 0, X)

imputer = SimpleImputer(strategy='constant', fill_value=0)
X = imputer.fit_transform(X)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled_df = pd.DataFrame(X_scaled, columns=pd.DataFrame(X).columns)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_scaled_df, y)

feature_importances = pd.DataFrame({'Feature': X_scaled_df.columns, 'Importance': rf.feature_importances_})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

print(feature_importances.head(10))


    Feature  Importance
42       42    0.054726
41       41    0.053957
39       39    0.052657
54       54    0.045941
0         0    0.045515
63       63    0.040766
52       52    0.037657
10       10    0.037336
4         4    0.034248
13       13    0.033366


In [22]:
feature_importances.to_csv('i.csv')

In [23]:
max_importance = feature_importances['Importance'].max()
feature_importances['Normalized_Importance'] = feature_importances['Importance'] / max_importance

threshold = 0.05  
threshold_value = threshold * max_importance

important_features = feature_importances[feature_importances['Importance'] >= threshold_value]

features_to_remove = feature_importances[feature_importances['Importance'] < threshold_value]

print("Features to keep:")
print(important_features)

print("\nFeatures to remove:")
print(features_to_remove)

Features to keep:
    Feature  Importance  Normalized_Importance
42       42    0.054726               1.000000
41       41    0.053957               0.985951
39       39    0.052657               0.962195
54       54    0.045941               0.839467
0         0    0.045515               0.831691
63       63    0.040766               0.744916
52       52    0.037657               0.688101
10       10    0.037336               0.682233
4         4    0.034248               0.625806
13       13    0.033366               0.609690
6         6    0.028405               0.519047
66       66    0.027320               0.499223
5         5    0.026142               0.477686
12       12    0.024007               0.438683
8         8    0.021852               0.399301
40       40    0.021654               0.395680
65       65    0.021401               0.391061
53       53    0.021069               0.384990
55       55    0.019286               0.352418
69       69    0.017068               0.31

In [25]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt



# Normalize feature importance (optional)
max_importance = feature_importances['Importance'].max()
feature_importances['Normalized_Importance'] = feature_importances['Importance'] / max_importance

# Loop to check multiple thresholds
thresholds = np.linspace(0.05, 0.5, 10)  # Thresholds from 5% to 50%
best_score = -np.inf
best_threshold = None
best_model = None
best_features = None

for threshold in thresholds:
    # Determine the importance threshold
    threshold_value = threshold * max_importance

    # Select features above the threshold
    selected_features = feature_importances[feature_importances['Importance'] >= threshold_value]['Feature']
    
    # Filter dataset to use only selected features
    X_selected = X_scaled_df[selected_features]

    # Split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.3, random_state=42)
    
    # Train Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model with cross-validation (use accuracy or another metric as needed)
    cv_score = np.mean(cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy'))
    
    # If the current model performs better, save it
    if cv_score > best_score:
        best_score = cv_score
        best_threshold = threshold
        best_model = model
        best_features = selected_features

    print(f"Threshold: {threshold:.2f}, CV Score: {cv_score:.4f}")

# Output the best threshold and model performance
print(f"\nBest Threshold: {best_threshold:.2f}")
print(f"Best CV Score: {best_score:.4f}")
print(f"Selected Features: {best_features.tolist()}")


plt.plot(thresholds, best_score, marker='o', linestyle='-', color='b')
plt.xlabel("Threshold")
plt.ylabel("CV Accuracy")
plt.title("Model Performance vs Feature Importance Threshold")
plt.show()
